# Метод Кранка – Николсона для моделирования уравнения Шредингера

https://github.com/Aloys0/Schrodinger-equation

### Теория

Уравнение Шредингера имеет вид:

$$ i \hbar \frac{d}{dt} \psi(\vec{r}, t) = \hat{H}\psi(\vec{r}, t) \qquad (1)$$

Начальный волновой пакет Гаусса описывается как:

$$ \psi(x,0) = \mathrm{exp} \left(-\frac{(\vec{r}-\vec{r_0})^2}{2w^2} \right) \mathrm{exp} \left(i\vec{k} \cdot \vec{r} \right) \qquad (2)$$

Для моделирования мы берем $ \hbar = 2m = 1 $. Разностная аппроксимация уравнения Шредингера с использованием метода Кранка-Николсона становится:

$$ i \frac{\psi(\vec{r},t+\Delta t) - \psi(\vec{r},t)}{\Delta t} = \frac{\hat{H}\psi(\vec{r},t) + \hat{H}\psi(\vec{r},t+\Delta t)} {2} \qquad (3)$$

Которую можно переписать в виде:

$$ \left[1 + \frac{i\Delta t}{2}\hat{H}\right]\psi(\vec{r},t+\Delta t) = \left[1 - \frac{i\Delta t}{2}\hat{H}\right]\psi(\vec{r},t) \qquad (4)$$

#### 1D случай

В одномерном случае гамильтониан, действующий на волновую функцию, имеет вид:

$$ \hat{H}\psi(x,t) = - \frac{\psi(x+\Delta x,t) + \psi(x-\Delta x,t) - 2\psi(x,t)} {\Delta x^2} + V(x)\psi(x,t) \qquad (5)$$

Это соответствует разреженному гамильтониану. Например, для $ N_x = 5 $ гамильтониан и волновая функция становятся:

$$ H^{1D}=\begin{pmatrix}
-2 & 1 & 0 & 0 & 0\\
1 & -2 & 1 & 0 & 0\\
0 & 1 & -2 & 1 & 0\\
0 & 0 & 1 & -2 & 1\\
0 & 0 & 0 & 1 & -2
\end{pmatrix}, \qquad\qquad \psi^{1D}=\begin{pmatrix}
\psi_1\\
\psi_2\\
\psi_3\\
\psi_4\\
\psi_5
\end{pmatrix} $$


#### 2D случай 

Для двумерного случая гамильтониан, действующий на волновую функцию на равномерной сетке, имеет вид:

$$ \hat{H}\psi(x,t) = - \frac{\psi(x+\Delta x,y,t) + \psi(x-\Delta x,y,t) + \psi(x,y+\Delta y,t) + \psi(x,y+\Delta y,t) - 4\psi(x,y,t)} {\Delta x^2} + V(x,y)\psi(x,y,t) \qquad (6)$$

Для $ N_x= 3 $ гамильтониан и волновая функция будут выглядеть так:

$$ H^{2D}=\begin{pmatrix}
-4 & 1 & 0 & 1 & 0 & 0 & 0 & 0 & 0\\
1 & -4 & 1 & 0 & 1 & 0 & 0 & 0 & 0\\
0 & 1 & -4 & 0 & 0 & 1 & 0 & 0 & 0\\
1 & 0 & 0 & -4 & 1 & 0 & 1 & 0 & 0\\
0 & 1 & 0 & 1 & -4 & 1 & 0 & 1 & 0\\
0 & 0 & 1 & 0 & 1 & -4 & 0 & 0 & 1\\
0 & 0 & 0 & 1 & 0 & 0 & -4 & 1 & 0\\
0 & 0 & 0 & 0 & 1 & 0 & 1 & -4 & 1\\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 1 & -4
\end{pmatrix}, \qquad\qquad \psi^{2D}=\begin{pmatrix}
\psi_{11}\\
\psi_{21}\\
\psi_{31}\\
\psi_{12}\\
\psi_{22}\\
\psi_{32}\\
\psi_{13}\\
\psi_{23}\\
\psi_{33}\\
\end{pmatrix} $$

# Sparse

Используем операции над разреженными матрицами, хорошо оптимизированные в библиотеке `Sparse.jl`
Проиллюстрируем работу функций и сравним их производительность

In [ ]:
]add BenchmarkTools

In [ ]:
]add NumericalIntegration

In [1]:
# для нормировки
using NumericalIntegration # https://github.com/dextorious/NumericalIntegration.jl 

In [1]:
using LinearAlgebra, SparseArrays#, BenchmarkTools
# https://docs.julialang.org/en/v1/stdlib/SparseArrays/#Sparse-matrix-operations-1

In [3]:
V0 = diagm([-5:4;])

10×10 Array{Int64,2}:
 -5   0   0   0   0  0  0  0  0  0
  0  -4   0   0   0  0  0  0  0  0
  0   0  -3   0   0  0  0  0  0  0
  0   0   0  -2   0  0  0  0  0  0
  0   0   0   0  -1  0  0  0  0  0
  0   0   0   0   0  0  0  0  0  0
  0   0   0   0   0  0  1  0  0  0
  0   0   0   0   0  0  0  2  0  0
  0   0   0   0   0  0  0  0  3  0
  0   0   0   0   0  0  0  0  0  4

In [4]:
V1 = sparse(I, 10, 10).*[-5:4;]

10×10 SparseMatrixCSC{Int64,Int64} with 9 stored entries:
  [1 ,  1]  =  -5
  [2 ,  2]  =  -4
  [3 ,  3]  =  -3
  [4 ,  4]  =  -2
  [5 ,  5]  =  -1
  [7 ,  7]  =  1
  [8 ,  8]  =  2
  [9 ,  9]  =  3
  [10, 10]  =  4

In [5]:
A0 = SymTridiagonal( fill(-2.0, 10), ones(9) )

10×10 SymTridiagonal{Float64,Array{Float64,1}}:
 -2.0   1.0    ⋅     ⋅     ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
  1.0  -2.0   1.0    ⋅     ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅    1.0  -2.0   1.0    ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅    1.0  -2.0   1.0    ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅    1.0  -2.0   1.0    ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅    1.0  -2.0   1.0    ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅    1.0  -2.0   1.0    ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅    1.0  -2.0   1.0    ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅     ⋅    1.0  -2.0   1.0
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅     ⋅     ⋅    1.0  -2.0

In [6]:
A1 = spdiagm(-1 => ones(9), 0 => fill(-2.0, 10), 1 => ones(9) )

10×10 SparseMatrixCSC{Float64,Int64} with 28 stored entries:
  [1 ,  1]  =  -2.0
  [2 ,  1]  =  1.0
  [1 ,  2]  =  1.0
  [2 ,  2]  =  -2.0
  [3 ,  2]  =  1.0
  [2 ,  3]  =  1.0
  [3 ,  3]  =  -2.0
  [4 ,  3]  =  1.0
  [3 ,  4]  =  1.0
  [4 ,  4]  =  -2.0
  [5 ,  4]  =  1.0
  [4 ,  5]  =  1.0
  ⋮
  [6 ,  6]  =  -2.0
  [7 ,  6]  =  1.0
  [6 ,  7]  =  1.0
  [7 ,  7]  =  -2.0
  [8 ,  7]  =  1.0
  [7 ,  8]  =  1.0
  [8 ,  8]  =  -2.0
  [9 ,  8]  =  1.0
  [8 ,  9]  =  1.0
  [9 ,  9]  =  -2.0
  [10,  9]  =  1.0
  [9 , 10]  =  1.0
  [10, 10]  =  -2.0

In [7]:
A1 + -2I + zeros(10,10)

10×10 Array{Float64,2}:
 -4.0   1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
  1.0  -4.0   1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
  0.0   1.0  -4.0   1.0   0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   1.0  -4.0   1.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   1.0  -4.0   1.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   1.0  -4.0   1.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   1.0  -4.0   1.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0   1.0  -4.0   1.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0  -4.0   1.0
  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0  -4.0

`I` - единичная матрица специального типа. Она не имеет фиксированного размера и оптимизирована под разного рода операции

In [17]:
function foo1(N)
    C = [-N÷2:N÷2-1;]
    V = diagm(C)
    A = SymTridiagonal( fill(-2.0, N), ones(N-1) )
    2(A+V)\C
end

function foo2(N)
    C = [-N÷2:N÷2-1;]
    V = spdiagm(0 => C )
    A = SymTridiagonal( fill(-2.0, N), ones(N-1) )
    2(A+V)\C
end

function foo3(N)
    C = [-N÷2:N÷2-1;]
    V =  spdiagm(0 => C )
    A = spdiagm(-1 => ones(N-1), 0 => fill(-2.0, N), 1 => ones(N-1) )
    2(A+V)\C
end

foo3 (generic function with 1 method)

In [24]:
foo1(6), foo2(6), foo3(6)

([0.4, 0.5, 0.6, 0.8, 1.0, 0.7], [0.4, 0.5, 0.6, 0.8, 1.0, 0.7000000000000001], [0.4, 0.5, 0.6, 0.8, 1.0, 0.7000000000000001])

In [25]:
@benchmark foo1(1000)

BenchmarkTools.Trial: 
  memory estimate:  30.56 MiB
  allocs estimate:  18
  --------------
  minimum time:     43.742 ms (0.00% GC)
  median time:      47.869 ms (6.13% GC)
  mean time:        54.714 ms (8.67% GC)
  maximum time:     182.546 ms (7.10% GC)
  --------------
  samples:          92
  evals/sample:     1

In [26]:
@benchmark foo2(1000)

BenchmarkTools.Trial: 
  memory estimate:  923.28 KiB
  allocs estimate:  121
  --------------
  minimum time:     4.471 ms (0.00% GC)
  median time:      4.576 ms (0.00% GC)
  mean time:        4.743 ms (2.06% GC)
  maximum time:     65.942 ms (92.53% GC)
  --------------
  samples:          1053
  evals/sample:     1

In [27]:
@benchmark foo3(1000)

BenchmarkTools.Trial: 
  memory estimate:  983.30 KiB
  allocs estimate:  128
  --------------
  minimum time:     631.967 μs (0.00% GC)
  median time:      700.651 μs (0.00% GC)
  mean time:        817.506 μs (5.46% GC)
  maximum time:     62.658 ms (97.12% GC)
  --------------
  samples:          6082
  evals/sample:     1

# Solve

Вывод: разреженные матрицы это чертовски удобно. Теперь соберем все вместе

In [2]:
function Potential(potential)
    # Определите потенциал V(x) в соответствии 
    # с конкретной конфигурацией, заданной как «potential».
    
    V = zeros(Nx)
    
    if potential == "infinite barrier"
        a = round(Int, 0.7*Nx)
        V[a:end] .= 1e10
    
    elseif potential == "infinite well"
        a = round(Int, 0.4*Nx)
        b = round(Int, 0.6*Nx)
        
        V[1:a] .= 1e10
        V[b:end] .= 1e10
        
    elseif potential == "tunneling"
        a = round(Int, 0.7*Nx)
        b = round(Int, 0.75*Nx)
        V[a:b] .= 1e4
    
    elseif potential == "harmonic oscillator"
        V .= 2e5*x.^2
    else
        println("Not a compatible potential, V=0")
    end
    
    return V
end

Potential (generic function with 1 method)

In [45]:
function simulate(V)
    # Вычислить волновую функцию psi методом
    # Кранка-Николсона с граничными условиями Дирихле.
    
    psi = zeros(Complex, Nx, Nt)
    
    # Гамильтониан как разреженная матрица
    Vm = spdiagm(0 => V )
    H = spdiagm(-1 => ones(Nx-1), 0 => fill(-2.0, Nx), 1 => ones(Nx-1) )
    H /= dx^2
    H += Vm

    A = 0.5im*dt * H + I # Левая матрица в уравнении (4)

    gauss_x(x, gaussWidth, x0, waveVector) = exp(-(x-x0)^2/(2*gaussWidth^2)) * exp(im*x*waveVector)
    
    
    psi[:,1] = gauss_x.(x, gaussWidth, x0, waveVector)
    #psi /= np.trapz(abs(psi[0])**2, dx=dx) # Normalisation
    
    Y = abs2.(psi[:,1])
    X = range(0, step = 0.1, length = length(Y)) |> collect
    #psi[:,1] /= integrate(X, Y ) # Normalisation

    for t = 1:Nt-1
        
        b = ( I - 0.5im*dt * H ) * psi[:,t] #
        # правая часть уравнения (4)
        
        # Использует встроенные солверы для решения СЛАУ A * psi = b
        psi[:,t+1] = A \ b
    end
    return abs2.( psi )
end

simulate (generic function with 1 method)

In [46]:
dx = 0.001 # Spacing between x values
dt = 0.0005 # Spacing between time steps

x = [-2:dx:2;]

Nx = length(x) # Number of spatial steps
Nt = 80 # Number of time steps

# Выберите один из потенциалов: infinite well, infinite barrier, tunneling, harmonic oscillator
V = Potential("tunneling")

# Parameters for the initial gaussian wave packet at t=0
gaussWidth = 0.08
waveVector = 0.
x0 = 0.0 # Initial position

@time psi = simulate(V);

  0.911226 seconds (5.71 M allocations: 638.575 MiB, 11.17% gc time)


In [9]:
using Gnuplot

┌ Info:   Gnuplot version: 5.2.0
└ @ Gnuplot C:\Users\User\.julia\packages\Gnuplot\GkEY3\src\Gnuplot.jl:111


In [52]:
@gp x psi[:, 60] "with lines notit lw 2 lc rgb 'black'"

In [13]:
cd("C:/Users/User/Desktop/Mycop")

In [29]:
# animation
frame = 0
for i in 1:80
    global frame += 1
    @gp :- frame x psi[:, i] "with lines notit lw 2 lc rgb 'black'" :-
end
@gp


save(term="gif animate size 480,360 delay 15", output="aanimation.gif")

GNUPLOT (default) -> wxt
GNUPLOT (default) -> 0 title "Gnuplot.jl: default" enhanced


"aanimation.gif"

GNUPLOT (default) -> 80 frames in animation sequence


# 2d

In [14]:
function potential2d(potntl, Nx)
    # Определите потенциал V (x) в соответствии 
    # с конкретной конфигурацией, заданной как «potential».
    V2D = zeros(Nx, Nx)
    a = round(Int, 0.3*Nx)
    b = round(Int, 0.4*Nx)
    c = round(Int, 0.6*Nx)
    d = round(Int, 0.7*Nx)
    e = round(Int, 0.6*Nx)
    f = round(Int, 0.65*Nx)
    
    if potntl == "infinite barrier"
        V2D[:, d:end] .= 1e10
    
    elseif potntl == "double slit"
        
        V2D[1:a,   e:f] .= 1e10
        V2D[b:c,   e:f] .= 1e10
        V2D[d:end, e:f] .= 1e10
    
    elseif potntl == "tunneling"
        V2D[:, e:f] .= 1e7
    
    elseif potntl == "harmonic oscillator"
        V2D = 2e5*(xx.^2 + yy.^2)
    
    else
        print("Not a compatible potential, V=0")
    end
    return V2D
end

potential2d (generic function with 2 methods)

Поиграемся с промежуточными результатами, чтоб понять, как они работают

In [6]:
V = zeros(3, 3)
V[2,:] .= 3
reshape(V, 9)

9-element Array{Float64,1}:
 0.0
 3.0
 0.0
 0.0
 3.0
 0.0
 0.0
 3.0
 0.0

In [ ]:
?kron

In [7]:
Nx = 3
I1 = diagm(ones(Nx))
I2 = diagm(ones(Nx^2))
H = spdiagm(-1 => ones(Nx-1), 0 => fill(-2.0, Nx), 1 => ones(Nx-1) )
kron(H, I1) + kron(I1, H) + diagm(reshape(V, 9))

9×9 Array{Float64,2}:
 -4.0   1.0   0.0   1.0   0.0   0.0   0.0   0.0   0.0
  1.0  -1.0   1.0   0.0   1.0   0.0   0.0   0.0   0.0
  0.0   1.0  -4.0   0.0   0.0   1.0   0.0   0.0   0.0
  1.0   0.0   0.0  -4.0   1.0   0.0   1.0   0.0   0.0
  0.0   1.0   0.0   1.0  -1.0   1.0   0.0   1.0   0.0
  0.0   0.0   1.0   0.0   1.0  -4.0   0.0   0.0   1.0
  0.0   0.0   0.0   1.0   0.0   0.0  -4.0   1.0   0.0
  0.0   0.0   0.0   0.0   1.0   0.0   1.0  -1.0   1.0
  0.0   0.0   0.0   0.0   0.0   1.0   0.0   1.0  -4.0

И приступим к моделированию:

In [3]:
function gauss2d(x, y, wx, wy, x0, y0, kx0, ky0)
    exp(-0.5*((x - x0)/wx)^2 -0.5*((y - y0)/wy)^2 + im*x*kx0 + im*y*ky0) * (hypot(wx, wy) * sqrt(pi))^-0.5
end

gauss2d (generic function with 1 method)

In [15]:
function simulation2D()
    # Calculate the wave function psi according to the Crank-Nicolson method with Dirichlet boundary conditions.
    
    dx = 0.05 # Spacing between x values
    dt = 0.0005 # Spacing between time steps

    X = [-1.5:dx:1.5;] # [-1.:dx:1.;]
    Y = copy(X)

    Nx = length(X) # Number of spatial steps
    Nt = 90 # Number of time steps

    # Choose one of the potentials: infinite barrier, double slit, tunneling, harmonic oscillator
    V2D = potential2d("tunneling", Nx)

    # Parameters for the initial gaussian wave packet at t=0
    width_x = 0.3 # 0.2
    width_y = 0.3 # 0.6
    wavect_x = -20
    wavect_y = 0
    x0 = -0.4 # Initial x-position
    y0 = 0 # Initial y-position
    
    Ψ = zeros( Complex, Nt, Nx, Nx)
    # 1D Hamiltonian as a sparse matrix
    
    I1 = spdiagm( 0 => ones(Nx) )
    I2 = spdiagm( 0 => ones(Nx^2) )
    H = spdiagm( -1 => ones(Nx-1), 0 => fill(-2.0, Nx), 1 => ones(Nx-1) )
    H /= dx^2
    H2D = kron(H, I1) + kron(I1, H) + diagm( reshape(V2D, Nx^2) ) # 2D Hamiltonian
    
    A = I2 + 0.5im*dt * H2D # Left hand side matrix in eq. (4)
    B = I2 - 0.5im*dt * H2D
    Ψ[1,:,:] .= [ gauss2d(x, y, width_x, width_y, x0, y0, wavect_x, wavect_y) for y in Y, x in X ]
    
    #Y = abs2.(psi[1,:,:])
    #X = range(0, step = 0.1, length = length(Y)) |> collect
    #psi[:,1] /= integrate(X, Y ) # Normalisation

    #psi[0] /= np.trapz(np.trapz(abs(psi2D[0])**2, dx=dx), dx=dx) # Normalisation
    #psi = psi.reshape((Nt, -1), order="F")
    
    Ψ = reshape(Ψ, Nt, Nx^2)
    for t = 1:Nt-1
        
        b = complex.(B * Ψ[t,:]) # правая часть уравнения (4)
        # Использует встроенные солверы для решения СЛАУ A * psi = b
        Ψ[t+1,:] .= A \ b
    end
    
    Ψ = reshape(Ψ, Nt, Nx, Nx)
    return abs2.(Ψ)
end

simulation2D (generic function with 1 method)

In [ ]:
@time psi2D = simulation2D();

In [10]:
@gp psi2D[30,:,:] "w image notit" "set auto fix"

┌ Info: Creating session default...
└ @ Gnuplot C:\Users\User\.julia\packages\Gnuplot\GkEY3\src\Gnuplot.jl:173
┌ Info:   Gnuplot version: 5.2.0
└ @ Gnuplot C:\Users\User\.julia\packages\Gnuplot\GkEY3\src\Gnuplot.jl:111


GNUPLOT (default) -> wxt
GNUPLOT (default) -> 0 enhanced


In [11]:
cd("C:\\Users\\User\\Desktop\\Mycop")

In [12]:
size(psi2D, 1)

90

In [13]:
# animation
frame = 0
for i in 1:size(psi2D, 1)
    global frame += 1
    @gp :- frame psi2D[i,:,:] "w image notit" "set auto fix" :-
end
@gp


save(term="gif animate size 480,360 delay 15", output="aanimation2.gif")

GNUPLOT (default) -> wxt
GNUPLOT (default) -> 0 title "Gnuplot.jl: default" enhanced


"aanimation.gif"

GNUPLOT (default) -> 90 frames in animation sequence
